LIMESODA - Liming Soil Dataset

# RAVEN

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from scipy.sparse import issparse
import networkx as nx
from itertools import combinations
import openml
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.datasets import fetch_openml
import numpy as np

In [2]:
def raven(data, mode='openml', sample_size=50, tau=0.95, target_col=None):
    """
    Implements the Raven algorithm that identifies redundant features in a dataset.

    Args: 
        data: DataFrame object, OpenML dataset ID (int), or name (str).
        mode (str): 'openml' (default) or 'df'. 
                    Specifies how to interpret the 'data' argument.
        tau (float): Threshold for correlation coefficient. Default is 0.95.
        sample_size (int): Number of samples to use. Default is 50.
        target_col (str, optional): Target column to drop.

    Returns:
        essential (list): Names of selected (non-redundant) features.
        redundant (list): Names of redundant features.
    """

    # --- Validate mode ---
    if mode not in ['openml', 'df']:
        raise ValueError("mode must be either 'openml' or 'df'")

    # --- Load dataset based on mode ---
    if mode == 'openml':
        if not isinstance(data, (int, str)):
            raise ValueError("If mode='openml', data must be an OpenML dataset ID (int) or name (str).")
        
        print(f"Fetching OpenML dataset: {data}...")
        dataset = openml.datasets.get_dataset(data)
        df, *_ = dataset.get_data(dataset_format="dataframe")
        if target_col is None and dataset.default_target_attribute:
            target_col = dataset.default_target_attribute
        if target_col and target_col in df.columns:
            df = df.drop(columns=[target_col])
        dataset_name = dataset.name


    elif mode == 'df':
        if not isinstance(data, pd.DataFrame):
            raise ValueError("If mode='df', data must be a pandas DataFrame.")
        
        df = data.copy()
        dataset_name = "Custom DataFrame"
        if target_col and target_col in df.columns:
            # Drop target column if specified for DataFrame
            df = df.drop(columns=[target_col])

    # --- Keep only numeric columns ---
    df = df.select_dtypes(include=[np.number])
    total_features = len(df.columns)

    # --- Validate parameters ---
    if tau <= 0 or tau >= 1:
        raise ValueError("tau must be greater than 0 and lesser than 1")
    if sample_size < 1:
        raise ValueError("sample_size must be greater than 0")
    if sample_size > len(df):
        print(f"Warning: sample_size ({sample_size}) is larger than dataset length ({len(df)}). Using full dataset (n={len(df)}) for sampling.")
        sample_size = len(df)
    if total_features < 2:
        raise ValueError("DataFrame must have at least 2 numeric columns")

    # --- Convert to numpy sample ---
    n_samples = min(sample_size, len(df))
    sample = df.sample(n_samples, random_state=42).to_numpy()
    r2_scores = {}
    col_idx = {col: df.columns.get_loc(col) for col in df.columns}

    # --- Compute R^2 between feature pairs ---

    for first, second in combinations(df.columns, 2):
        f_i, s_i = col_idx[first], col_idx[second]
        cov = np.cov(sample[:, f_i], sample[:, s_i])
        
        denom = cov[1, 1] * cov[0, 0]
        if denom == 0:
            r2_scores[first, second] = 0
        else:
            r2_scores[first, second] = cov[1, 0]**2 / denom

    # --- Build correlation graph ---

    def make_graph(scores, tau):
        def get_weight(r2): return (r2 - tau)/(1 - tau) * 0.5 + 0.5
        G = nx.Graph()
        for (a, b), r2 in scores.items():
            if r2 >= tau:
                G.add_edge(a, b, weight=get_weight(r2))
        return G

    G = make_graph(r2_scores, tau)
    del sample, r2_scores

    # --- Identify essential and redundant features ---

    essential = []
    for comp in nx.connected_components(G):
        sub = G.subgraph(comp)
        max_deg_node, _ = max(sub.degree(), key=lambda x: x[1])
        essential.append(max_deg_node)

    redundant = [node for node in G.nodes() if node not in essential]
    
    connected_features = set(G.nodes())
    all_features = set(df.columns)
    isolated = list(all_features - connected_features)
    essential = essential + isolated

    return essential, redundant

In [3]:
dataset = fetch_openml(data_id=47079, as_frame=False, parser='auto')
X_raw = dataset.data
y_raw = dataset.target
feature_names = np.array(dataset.feature_names)
print(f"Dataset loaded: {X_raw.shape[0]} samples, {X_raw.shape[1]} features.")

Dataset loaded: 138 samples, 2491 features.


In [4]:
if issparse(X_raw):
    print("Detected sparse matrix — converting to dense DataFrame for Raven...")
    X_dense_df = pd.DataFrame(X_raw.toarray(), columns=feature_names)
else:
    print("Detected dense NumPy array — creating DataFrame directly.")
    X_dense_df = pd.DataFrame(X_raw, columns=feature_names)

Detected dense NumPy array — creating DataFrame directly.


In [5]:
essential_features, redundant_features = raven(
    data=X_dense_df,
    mode='df',
    sample_size=200,  # small sample for speed
    tau=0.95
)
print(f"Raven selected {len(essential_features)} essential features out of {len(feature_names)}.")

Raven selected 3 essential features out of 2491.


In [ ]:
X_selected = X_dense_df[essential_features]
X_train, X_test, y_train, y_test = train_test_split(X_dense_df, y_raw, test_size=0.2, random_state=42)

rf_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor(
        n_estimators=100,
        random_state=42,
        n_jobs=-1
    ))
])


rf_pipeline.fit(X_train, y_train)
y_pred = rf_pipeline.predict(X_test)
print("\n---Results ---")
print(f"R² Score: {r2_score(y_test, y_pred):.4f}")
print(f"MSE: {mean_squared_error(y_test, y_pred):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.4f}")

# LASSO

In [4]:
dataset = fetch_openml(data_id=47079, as_frame=False, parser='auto')
    
X_raw = dataset.data
y_raw = dataset.target
feature_names = np.array(dataset.feature_names)
    
print(f"Dataset loaded: {X_raw.shape[0]} samples, {X_raw.shape[1]} features.")

Dataset loaded: 138 samples, 2491 features.


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X_raw, y_raw, test_size=0.3, random_state=42
)

is_sparse_input = issparse(X_train)
lasso_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler(with_mean=not is_sparse_input)),
        ('lasso', LassoCV(cv=5, random_state=42, n_jobs=-1, max_iter=10000))
])
lasso_pipeline.fit(X_train, y_train)
print("Model training complete.")

c:\Users\shrey\Desktop\RAVEN\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.153e-02, tolerance: 9.348e-03
  model = cd_fast.enet_coordinate_descent(
c:\Users\shrey\Desktop\RAVEN\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.287e-02, tolerance: 1.287e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\shrey\Desktop\RAVEN\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duali

Model training complete.


c:\Users\shrey\Desktop\RAVEN\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.904e-02, tolerance: 1.488e-02
  model = cd_fast.enet_coordinate_descent(


In [7]:
y_pred = lasso_pipeline.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
lasso_model = lasso_pipeline.named_steps['lasso']
selected_mask = np.abs(lasso_model.coef_) > 1e-6
selected_count = np.sum(selected_mask)

print("\n--- LASSO Results ---")
print(f"R² Score: {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"Selected Features: {selected_count} / {X_raw.shape[1]}")



--- LASSO Results ---
R² Score: 0.9115
MSE: 0.1127
MAE: 0.2842
Selected Features: 20 / 2491


# WITHOUT RAVEN or LASSO

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X_raw, y_raw, test_size=0.3, random_state=42
)
is_sparse_input = issparse(X_train)

rf_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler(with_mean=not is_sparse_input)),
        ('model', RandomForestRegressor(
            n_estimators=100, 
            random_state=42, 
            n_jobs=-1  # Use all CPU cores
    ))
])
rf_pipeline.fit(X_train, y_train)
print("Model training complete.")

Model training complete.


In [9]:
y_pred = rf_pipeline.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"R² Score: {r2:.4f}")
print(f"MSE:      {mse:.4f}")
print(f"MAE:      {mae:.4f}")
print(f"Features Used: {X_raw.shape[1]} (All)")

R² Score: 0.7429
MSE:      0.3273
MAE:      0.4691
Features Used: 2491 (All)
